In [1]:
import os
import json
import glob

In [2]:
data_root = 'multiwoz/HDSA'

In [3]:
glob.glob(data_root + '/*')

['multiwoz/HDSA/results.txt.pred.non_delex',
 'multiwoz/HDSA/results.txt.pred.BERT_dim128_w_domain.pred',
 'multiwoz/HDSA/results.txt.non_delex']

In [5]:
!head multiwoz/HDSA/results.txt.pred.BERT_dim128_w_domain.pred

{
  "MUL0003": [
    "there are [value_count] hotel -s that meet your criteria . is there a particular area you would like to stay in ?",
    "[hotel_name] is in the [hotel_pricerange] price range . would you like me to book it for you ?",
    "i would recommend the [hotel_name] . would you like me to book it for you ?",
    "what day will you be arriving and how many nights will you be staying ?",
    "would you like me to book it for you ?",
    "yes , it is . would you like me to book it for you ?",
    "booking was successful . the table will be reserved for [value_count] minutes . reference number is [restaurant_reference] . is there anything else i can help you with ?",
    "you are welcome . have a great day !"
